Hypercube visibility bound
==============
by excluding structures with size up to 4

In [1]:
HV = HypercubeVertexTheory
from collections import deque
def _bits_set_positions(mask, d):
    return [i for i in range(d) if (mask >> i) & 1]
def geodesic_path_exists(d, x, y, S):
    if x == y:
        return True, [x]
    diff_mask = x ^^ y
    diff_positions = _bits_set_positions(diff_mask, d)
    h = len(diff_positions) 
    pos_to_bit = {pos: (1 << idx) for idx, pos in enumerate(diff_positions)}
    local_to_vertex = [None] * (1 << h)
    for local in range(1 << h):
        real_mask = 0
        for idx, pos in enumerate(diff_positions):
            if (local >> idx) & 1:
                real_mask |= (1 << pos)
        local_to_vertex[local] = x ^^ real_mask
    forbidden = set()
    for local, v in enumerate(local_to_vertex):
        if local != 0 and local != (1 << h) - 1 and v in S:
            forbidden.add(local)
    start = 0
    goal = (1 << h) - 1
    if start in forbidden or goal in forbidden:
        forbidden.discard(start)
        forbidden.discard(goal)
    prev = {start: None}
    q = deque([start])
    while q:
        cur = q.popleft()
        if cur == goal:
            break
        for idx in range(h):
            if not ((cur >> idx) & 1):
                nxt = cur | (1 << idx)
                if nxt in forbidden or nxt in prev:
                    continue
                prev[nxt] = (cur, idx)
                q.append(nxt)
    if goal not in prev and goal != start:
        return False, []
    path_local = []
    cur = goal
    while cur is not None:
        path_local.append(cur)
        cur = prev[cur][0] if prev[cur] is not None else None
    path_local.reverse()
    path_vertices = [local_to_vertex[local] for local in path_local]
    return True, path_vertices
def test_hv(hv):
    d = int(log(hv.size(), 2))
    S = [xx[0] for xx in hv.blocks("points")]
    for xx,yy in itertools.combinations(S, 2):
        res, path = geodesic_path_exists(d, xx, yy, S)
        if not res:
            return False, (xx, yy)
    return True, None

In [2]:
HV.reset()
excls = {}
for dd in range(2, 5):
    print("dimension", dd)
    excldd = []
    for hv in HV.generate(2**dd):
        S = [xx[0] for xx in hv.blocks("points")]
        res, pair = test_hv(hv)
        if not res:
            excldd.append(hv)
            print(" - excluding", [yy[0] for yy in hv.blocks("points")])
    HV.exclude(excldd)
    excls[dd] = excldd

dimension 2
 - excluding [0, 1, 2, 3]
dimension 3
 - excluding [0, 1, 2, 4, 7]
 - excluding [0, 1, 2, 5, 6, 7]
dimension 4
 - excluding [0, 1, 2, 4, 8, 15]
 - excluding [0, 1, 2, 4, 9, 10, 12, 15]
 - excluding [0, 1, 2, 4, 11, 13, 14, 15]
 - excluding [0, 3, 5, 6, 9, 10, 12, 15]
 - excluding [0, 1, 2, 4, 9, 10, 13, 14, 15]


In [6]:
HV.printlevel(0)
bound = HV.optimize(HV.generate(1)[1], 16, exact=True, denom=2**20, file="certificate")
print("Found bound is {} ~ {}".format(bound, bound.n()))

Found bound is 1916879/4718592 ~ 0.406239615546332
